# 앙상블(Ensemble) 학습

In [1]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

In [2]:
from sklearn.preprocessing import MinMaxScaler
cancer_scaled = MinMaxScaler().fit_transform(cancer.data)

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    cancer_scaled, cancer.target, stratify=cancer.target, test_size=0.2, random_state=2021
)

### 1. 앙상블 학습 - Hard Voting
- 로지스틱 회귀
- 서포트 벡터 머신
- K 최근접 이웃

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [5]:
lrc = LogisticRegression()
svc = SVC()
knn = KNeighborsClassifier()

In [6]:
from sklearn.ensemble import VotingClassifier

voc = VotingClassifier(
    estimators=[('LRC',lrc),('SVC',svc),('KNN',knn)], voting='hard'
)

In [7]:
voc.fit(X_train, y_train)
voc.score(X_test, y_test)

0.9824561403508771

- 개별 분류기의 성능

In [8]:
lrc.fit(X_train, y_train)
svc.fit(X_train, y_train)
knn.fit(X_train, y_train)
lrc.score(X_test, y_test), svc.score(X_test, y_test), knn.score(X_test, y_test)

(0.9824561403508771, 0.9824561403508771, 0.9824561403508771)

### 2. 앙상블 학습 - 소프트 보팅

In [9]:
# 메소드와 속성
print(dir(lrc))

['C', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_check_n_features', '_estimator_type', '_get_param_names', '_get_tags', '_more_tags', '_predict_proba_lr', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_validate_data', 'class_weight', 'classes_', 'coef_', 'decision_function', 'densify', 'dual', 'fit', 'fit_intercept', 'get_params', 'intercept_', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_features_in_', 'n_iter_', 'n_jobs', 'penalty', 'predict', 'predict_log_proba', 'predict_proba', 'random_state', 'score', 'set_params', 'solver', 'sparsify', 'tol', 'verbose', 'warm_start']


In [10]:
lrc.predict_proba(X_test[:5])

array([[0.36348222, 0.63651778],
       [0.97162943, 0.02837057],
       [0.1869565 , 0.8130435 ],
       [0.02133462, 0.97866538],
       [0.0548391 , 0.9451609 ]])

In [11]:
lrc.predict(X_test[:5])

array([1, 0, 1, 1, 1])

In [12]:
svc.predict_proba(X_test[:5])

AttributeError: predict_proba is not available when  probability=False

In [13]:
svc.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [14]:
svc2 = SVC(probability=True)
svc2.fit(X_train, y_train)
svc2.predict_proba(X_test[:5])

array([[4.64891897e-01, 5.35108103e-01],
       [9.99845588e-01, 1.54412379e-04],
       [1.48574490e-02, 9.85142551e-01],
       [3.10319203e-03, 9.96896808e-01],
       [4.48756819e-03, 9.95512432e-01]])

In [15]:
knn.predict_proba(X_test[:5])

array([[0.4, 0.6],
       [1. , 0. ],
       [0.2, 0.8],
       [0. , 1. ],
       [0. , 1. ]])

In [16]:
voc2 = VotingClassifier(
    estimators=[('LRC',lrc),('SVC',svc2),('KNN',knn)], voting='soft'
)

In [17]:
voc2.fit(X_train, y_train)
voc2.score(X_test, y_test)

0.9912280701754386

In [18]:
voc2.predict_proba(X_test[:5])

array([[0.40787161, 0.59212839],
       [0.99048643, 0.00951357],
       [0.13401809, 0.86598191],
       [0.00817742, 0.99182258],
       [0.01981607, 0.98018393]])

- GridSearchCV 적용

In [19]:
params = {
    'LRC__C': [0.1,1,10],
    'SVC__C': [0.1,1,10]
}

In [20]:
from sklearn.model_selection import GridSearchCV
grid_voc2 = GridSearchCV(voc2, param_grid=params, scoring='accuracy', cv=3)
grid_voc2.fit(X_train, y_train)
grid_voc2.best_params_

{'LRC__C': 0.1, 'SVC__C': 10}

In [22]:
params = {
    'LRC__C': [0.01,0.05,0.1,0.2,0.5],
    'SVC__C': [5,10,20,50]
}
grid_voc2 = GridSearchCV(voc2, param_grid=params, scoring='accuracy', cv=3)
grid_voc2.fit(X_train, y_train)
grid_voc2.best_params_

{'LRC__C': 0.01, 'SVC__C': 50}

In [23]:
grid_voc2.best_estimator_.score(X_test, y_test)

0.9912280701754386

### 3. Random Forest
- DecisionTree 100개

In [24]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=2021)
rfc.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 2021,
 'verbose': 0,
 'warm_start': False}

In [25]:
rfc.fit(X_train, y_train)
rfc.score(X_test, y_test)

0.9736842105263158